In [1]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split, cross_val_score
# from tpot.config.classifier_nn import classifier_config_nn

from sklearn.pipeline import make_pipeline
# from tpot.config import classifier_config_dict_light
from tpot.config import classifier_config_dict
from sklearn.neighbors import KNeighborsClassifier

from keras import regularizers


import pandas as pd
import numpy as np
import os
import glob

import time

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
pd.options.mode.chained_assignment = None

C:\Users\monicai\AppData\Local\Continuum\anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\monicai\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# personal_config = classifier_config_dict_light
personal_config = classifier_config_dict
personal_config['tpot.builtins.SparseAutoencoder'] = {
    'regularizer':[regularizers.l1(10e-5)],
    'encoding_dim': [10, 50, 100, 500],
    'activation': ['relu'],
    'optimizer': ['adadelta'],
    'loss':['binary_crossentropy'],
    'epochs':[10],
    'batch_size':[10, 15]
}

In [3]:
#new data
new_data = pd.read_csv("processed_aml_data.csv")
X = new_data.iloc[:, 1:5051]
y = new_data.iloc[:, 5051]

In [4]:
#convert X and y to numpy
X = X.values
y = y.values

In [5]:
#split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(X, y, train_size = 0.8, test_size = 0.20, random_state = 42)
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_val = x_val.reshape((len(x_val)), np.prod(x_val.shape[1:]))

In [6]:
x_train.shape

(42, 5050)

In [7]:
y_train.shape

(42,)

In [8]:
from tpot.builtins import SparseAutoencoder

In [12]:
sae = SparseAutoencoder(encoding_dim=40,
                        regularizer = regularizers.l1(10e-5),
                        activation='relu', 
                        optimizer='adadelta', 
                        loss='binary_crossentropy', 
                        epochs=10, batch_size=10)

In [13]:
sae.fit(x_train, y_train)

Train on 31 samples, validate on 11 samples
Epoch 1/10
31/31 [==============================] - ETA: 0s - loss: 0.718 - 0s 9ms/step - loss: 0.7141 - val_loss: 0.7117
Epoch 2/10
31/31 [==============================] - ETA: 0s - loss: 0.704 - 0s 2ms/step - loss: 0.7037 - val_loss: 0.7096
Epoch 3/10
31/31 [==============================] - ETA: 0s - loss: 0.698 - 0s 2ms/step - loss: 0.6984 - val_loss: 0.7080
Epoch 4/10
31/31 [==============================] - ETA: 0s - loss: 0.695 - 0s 2ms/step - loss: 0.6957 - val_loss: 0.7063
Epoch 5/10
31/31 [==============================] - ETA: 0s - loss: 0.693 - 0s 2ms/step - loss: 0.6931 - val_loss: 0.7031
Epoch 6/10
31/31 [==============================] - ETA: 0s - loss: 0.693 - 0s 2ms/step - loss: 0.6921 - val_loss: 0.7013
Epoch 7/10
31/31 [==============================] - ETA: 0s - loss: 0.691 - 0s 2ms/step - loss: 0.6903 - val_loss: 0.6994
Epoch 8/10
31/31 [==============================] - ETA: 0s - loss: 0.689 - 0s 2ms/step - loss: 0.6878

In [16]:
tpot = TPOTClassifier(generations=10, config_dict=personal_config,
                        population_size=10, verbosity=3,
                        template = 'SparseAutoencoder-RandomForestClassifier')

In [20]:
#start = time.time()
tpot.fit(x_train, y_train)
#end = time.time()
#print(end - start)

31 operators have been imported by TPOT.


RuntimeError: There was an error in the TPOT optimization process. This could be because the data was not formatted properly, or because data for a regression problem was provided to the TPOTClassifier object. Please make sure you passed the data to TPOT correctly.

In [ ]:
tpot.fitted_pipeline_

In [ ]:
tpot.score(x_train, y_train)

In [ ]:
tpot.score(x_val, y_val)